<a href="https://colab.research.google.com/github/Summarizer-cb/Gas_Energy_Summarization/blob/main/%5B%EC%B0%BD%EC%9A%B0%5D1206_FinetuningStep.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Load 

### Import 

In [1]:
!pip install transformers
!pip install wandb

     |████████████████████████████████| 3.1 MB 9.1 MB/s 
     |████████████████████████████████| 3.3 MB 74.4 MB/s 
     |████████████████████████████████| 895 kB 70.8 MB/s 
     |████████████████████████████████| 61 kB 557 kB/s 
     |████████████████████████████████| 596 kB 70.8 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
     |████████████████████████████████| 1.7 MB 8.3 MB/s 
     |████████████████████████████████| 140 kB 74.9 MB/s 
     |████████████████████████████████| 97 kB 8.5 MB/s 
     |████████████████████████████████| 180 kB 76.9 MB/s 
     |████████████████████████████████| 63 kB 2.0 MB/s 
  Created wheel for subprocess32: filename=subprocess32-3.5.4-py3-none-any.whl size=6502 sha256=28437ec4bf4fb27cc8a05961c613086e8ae827fa7e8a5a036826f814cacdf62f
  Stored in directory: /root/.cache/pip/wheels/50/ca/fa/8fca8d246e64f19488d07567547ddec8eb084e8c0d7a59226a
  Created w

In [2]:
import os
import math
import json
import time
import random
import multiprocessing
import json
import glob
import warnings
warnings.filterwarnings("ignore", category=UserWarning)

import numpy as np
import pandas as pd
from tqdm.auto import tqdm, trange
from sklearn import model_selection

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.nn import Parameter
import torch.optim as optim
from torch.utils.data.distributed import DistributedSampler
from torch.utils.data import (
    Dataset, DataLoader,
    SequentialSampler, RandomSampler
)
import transformers
from transformers import (AutoModel, AutoTokenizer, AdamW, BartForConditionalGeneration)


## Load dataset

### trainset

In [3]:
df_train = pd.read_csv("/<your_dir>/GSG_AIHUB_finetune_250000.csv")
train_set = df_train[['content',	'abstractive']]
train_set

,content,abstractive
0,전북일보 메가톤급 파장을 불러왔던 군산조선소와 GM대우 군산공장 폐쇄로 인해 가려져...,익산에서만 2400여명의 일자리가 사라지는 등 심각한 경제난을 겪어온 익산은 넥솔론...
1,한국과학기술정보연구원(KISTI)은 한국과학창의재단과 함께 '2019년도 학부생 연...,한국과학기술정보연구원(KISTI)은 교육부와 한국과학창의재단이 함께 진행하는 '20...
2,서울신문 올해도 최저임금 관련 논의가 파행을 겪고 있다.<SEP>어제는 2020 년...,2017년 대선에서 모든 후보가 '최저임금 1만원'을 약속한 것은 저임금 노동자 보...
3,태호군은 평소 축구하는 것을 몹시 좋아했다.<SEP>지난달 인천 '축구클럽 통학차 ...,22일 오전 인천 '축구클럽 통학차 사고'국민청원이 20만명 동의를 얻고 청와대로...
4,ᆞ가야금 명인의 음악적 발자취▲국악 2018 마스터피스-황병기일시 9월 18일~19...,"국악2018 마스터피스, 무위, 에쿠우스, 막스 어커만의 공연이 열린다."
...,...,...
249995,"동구, 대학원생들 아이디어로 관광콘텐츠 개발전남대 문화전문대학원생과 연구용역 발표회...",광주광역시 동구는 지난 3월 전남대학교 문화전문대학원과 동구 관광활성화를 위해 공동...
249996,엘리자베스 2세 영국 여왕의 차남인 앤드루 왕자가 14일 경북 안동시 서후면 봉정사...,엘리자베스 2세 여왕의 차남 앤드루왕자가 국화차를 최초로 만들고 대중화 한 봉정사에...
249997,나는 이래서 자영업을 포기했다매출마진 30%는 돼야 먹고사는 게 일반 장사치들인데 ...,"'일반 장사인들은 매출 마진 30%는 돼야 먹고사는데, 매출의 35%를 먼저 내야 ..."
249998,인천일보 2 조7000 억원이 투입된 경인아라뱃길의 기능 재정립을 위한 논의가 시작...,건설 당시 물류기능에 대해서 회의적이었던 반응을 무시한 채 공기업을 부실화시켜가며 ...


### valset

In [4]:
df_val = pd.read_csv("/<your_dir>/GSG_AIHUB_validate_10000.csv")
val_set = df_val[['content',	'abstractive']]
val_set

,content,abstractive
0,서울신문 부모가 된 10 대에게 쏟아지는 세상의 시선은 싸늘하다.<SEP>그 싸늘한...,10대 부모들이 겪고 있는 현실적 악순환은 기성세대에 사회적 연대 책임 역시 있으므...
1,김웅섭 지난 22일 이천 도예인들이 시청 앞 광장에서 도자기축제 행사에 돔 텐트를 ...,지난 22일 이천 도예인들이 시청 앞 광장에서 도자기축제 행사에 돔 텐트를 설치해 ...
2,또 향후 15년 간 경제 규모가 최대 9% 줄어들 것으로 예측됐다.<SEP>영국 정...,지난 26일 영국 정부가 발표한 노딜 브렉시트 시나리오를 따를 경우 경제에 끼치는 ...
3,성인남녀 3543명 대상 설문뉴시스성인남녀 10명 중 6명이 ‘유튜버’를 하고 싶어...,구인구직 매칭플랫폼 사람인(대표 김용환)이 성인남녀 3543명을 대상으로 조사한 결...
4,소득없는 배우자·경단여 등 이달부터 노후준비 서비스 제공 국민연금공단(이사장 김성주...,국민연금공단이 국민의 노후보장 강화를 위해 가입내역안내서 모바일 통지 대상자를 소득...
...,...,...
9995,31일 방송통신위원회는 정보통신정책연구원과 함께 지난해 12월 아세안 5개국 현지 ...,31일 방송통신위원회는 지난해 12월 아세안 5개국 국가별로 각 500명씩을 무작위...
9996,경북도·경주시·한국원자력연구원은 16 일 경주시 감포읍 나정리에 들어설 1 조원 규...,경북도·경주시·한국원자력연구원은 16 일 경주시 감포읍 나정리에 들어설 1조원 규모...
9997,부안군에 대한 전북도의 종합감사 결과 결원을 초과한 승진 인사를 임용하는 등 총 3...,"27일 전북도는 최근 부안군 종합감사에서 시정 8건, 주의 7건, 시정주의 14건,..."
9998,전북도는 6일 도내 바다 양식장 면적이 새만금사업 이전으로 회복됐다고 밝혔다.<SE...,"전북도는 6일 도내 바다 양식장 7,683ha가 폐업 보상되면서 수산업이 크게 위축..."


### testset

In [5]:
with open('/<your_dir>/new_test.jsonl', "r", encoding="utf-8") as f:
    jsonl = list(f)
test_data = []
for json_str in jsonl:
    test_data.append(json.loads(json_str))
len(test_data)


6596

In [6]:
dict_test = {'id':[],'content':[],'media':[]}
for data in test_data:
  dict_test['id'].append(data['id'])
  dict_test['content'].append(data['article_original'])
  dict_test['media'].append(data['media'])
test_set = pd.DataFrame(dict_test)
test_set

,id,content,media
0,472779091,[일본 사법당국의 출국금지 명령이 내려져 있던 카를로스 곤(65) 전 르노닛산 회장...,중앙일보
1,472779092,"[본선에서 트럼프에 패할 수 있다는 우려 때문이다., 지지율 3위: 엘리자베스 워런...",중앙일보
2,472779093,"[1월 1일 새해 김정은 북한 국무위원장은 모습을 드러내지 않았다., 2012년 집...",중앙일보
3,472779094,[국내 이동통신 3사(KT·SK텔레콤·LG유플러스)는 산토끼(신규 고객)를 잡느라 ...,중앙일보
4,472779095,[카를로스 곤(66)닛산자동차 전 회장의 ‘보석기간 중 탈출극’으로 인한 일본내 충...,중앙일보
...,...,...,...
6591,472785682,"[제1조(목적)"", ""이 고시는 「온실가스 배출권의 할당 및 거래에 관한 법률」(이...",환경부
6592,472785683,[산업통상자원부와 한국가스안전공사는 겨울철 불법 가스용품 유통에 따른 가스사고 예방...,헤럴드경제
6593,472785684,[정부가 올해 안에 2050 탄소중립 시나리오와 2030년 국가 온실가스감축목표(N...,머니투데이
6594,472785685,[정부가 수소차 보급 활성화를 위해 수소 제조용 천연가스 요금을 한시적으로 낮춘다....,경향신문


### submission

In [7]:
submission = pd.read_csv('/<your_dir>/new_sample_submission.csv')
submission

,id,summary
0,472779091,요약문
1,472779092,요약문
2,472779093,요약문
3,472779094,요약문
4,472779095,요약문
...,...,...
6591,472785682,요약문
6592,472785683,요약문
6593,472785684,요약문
6594,472785685,요약문


## DataLoader

In [8]:
tokenizer = AutoTokenizer.from_pretrained("hyunwoongko/kobart")
added_token_num = tokenizer.add_special_tokens({"additional_special_tokens":["<GSG>"]})
tokenizer.special_tokens_map

Downloading:   0%|          | 0.00/109 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.08k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/436k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/172k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/666k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.00 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/85.0 [00:00<?, ?B/s]

{'additional_special_tokens': ['<GSG>'],
 'bos_token': '<s>',
 'cls_token': '<s>',
 'eos_token': '</s>',
 'mask_token': '<mask>',
 'pad_token': '<pad>',
 'sep_token': '</s>',
 'unk_token': '<unk>'}

In [9]:
class TrainDataset(torch.utils.data.Dataset):
  def __init__(self,df,val=False):
    self.tokenizer = tokenizer
    self.df = df
    self.val = val
  
  def __getitem__(self,idx):
    doc = self.df.iloc[idx]
    input = doc['content'].replace('<SEP>',' ')
    label_text = doc['abstractive'] + "</s>"

    input_text = tokenizer.batch_encode_plus([input],padding='max_length',truncation=True, max_length=512, return_tensors="pt")
    label = tokenizer.batch_encode_plus([label_text],padding='max_length',truncation=True, max_length=512, return_tensors="pt")

    if self.val :
      return {"input_ids": input_text['input_ids'].flatten(), 
              "attention_mask": input_text['attention_mask'].flatten(), 
              "label": label['input_ids'].flatten(),
              "label_text": label_text
              }

    return {"input_ids": input_text['input_ids'].flatten(), 
            "attention_mask": input_text['attention_mask'].flatten(), 
            "label": label['input_ids'].flatten()
            }

  def __len__(self):
      return self.df.__len__()

  def __repr__(self):
    return f"data size : {self.__len__()} "

traindataset = TrainDataset(df_train)
valdataset = TrainDataset(df_val,val=True)

In [10]:
batch_size = 4
train_dataloader = DataLoader(traindataset, shuffle=True, batch_size=batch_size)
for sample in train_dataloader:
  print(sample)
  break

{'input_ids': tensor([[13724, 12037, 11842,  ..., 15053, 14141, 16827],
        [12141, 14697, 18004,  ...,     3,     3,     3],
        [15737, 12037, 14116,  ...,     3,     3,     3],
        [19132, 15530, 17002,  ..., 20780, 15358, 17492]]), 'attention_mask': tensor([[1, 1, 1,  ..., 1, 1, 1],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 1, 1, 1]]), 'label': tensor([[14762, 10667, 11908,  ...,     3,     3,     3],
        [ 9517,  9760, 14239,  ...,     3,     3,     3],
        [  238, 10746, 13282,  ...,     3,     3,     3],
        [11989, 13756,  9807,  ...,     3,     3,     3]])}


In [11]:
val_dataloader = DataLoader(valdataset, shuffle=False, batch_size=batch_size)
for sample in val_dataloader:
  print(sample)
  break

{'input_ids': tensor([[15295, 15955, 25596,  ...,     3,     3,     3],
        [ 9275, 11926, 11277,  ...,     3,     3,     3],
        [10109, 16090, 27712,  ...,     3,     3,     3],
        [11280, 12037,  9506,  ...,     3,     3,     3]]), 'attention_mask': tensor([[1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0]]), 'label': tensor([[15684,  9776, 15714,  ...,     3,     3,     3],
        [25560, 16670, 19989,  ...,     3,     3,     3],
        [25560, 16827, 15609,  ...,     3,     3,     3],
        [ 9170, 12037,  9170,  ...,     3,     3,     3]]), 'label_text': ['10대 부모들이 겪고 있는 현실적 악순환은 기성세대에 사회적 연대 책임 역시 있으므로, 이들의 학업권을 보호하고 육아를 병행할 수 있는 체계적인 사회적 지원을 제공하도록 노력해야한다.</s>', '지난 22일 이천 도예인들이 시청 앞 광장에서 도자기축제 행사에 돔 텐트를 설치해 참여 도예인들이 한 공간에서 행사를 치를 수 있도록 요구하는 집회를 벌였는데 시는 현재 예스파크 내에는 돔 텐트를 설치할 만큼의 공간이 없고, 지난 행사에서의 미흡한 점을 보완하고자 예스파크 내 공실 건물을 이용하자는 안을 제시하고 있다.</s>', '지난 26일 영국 정부가 발표한 노딜 브렉시트 시나리오를 따를 경우 경제

## Pretrained Model

In [ ]:
pretrained_weight = "/<your_dir>/KoPEGASUS_Finetuning_1_done_0.00000.pt"
pretrained_weight

In [13]:
model = BartForConditionalGeneration.from_pretrained("hyunwoongko/kobart",return_dict=True)
model.resize_token_embeddings(tokenizer.vocab_size + added_token_num)
# model.load_state_dict(torch.load(pretrained_weight, map_location=torch.device('cpu')))
model.load_state_dict(torch.load(pretrained_weight))  # GPU 사용시

Downloading:   0%|          | 0.00/473M [00:00<?, ?B/s]

<All keys matched successfully>

In [14]:
for bindx, batch in enumerate(val_dataloader):
  print("INPUT :",''.join(tokenizer.batch_decode(batch['input_ids'])[1]))
  print("PRED  :",''.join(tokenizer.batch_decode(model.generate(batch['input_ids'])[1])))
  print("LABEL :",batch['label_text'][1])
  break

INPUT : 김웅섭 지난 22일 이천 도예인들이 시청 앞 광장에서 도자기축제 행사에 돔 텐트를 설치해 줄 것을 요구하는 집회를 벌이고 있다. 이천도자기축제 기간 동안 행사장 내 텐트 설치를 놓고 도예인과 시가 의견 충동을 일으켜, 축제 준비에 빨간불이 들어왔다. 더욱이 축제(4월26~5월12일)가 60여일 앞으로 다가온 만큼, 시급한 대책이 요구되고 있다. 시 관내 도예인 50여 명은 지난 22일 시청 앞 광장에서 집회를 열고 "도자기축제가 도예인은 배제한 채 관 주도 행사로 전락했다. 도예인들의 의견은 수렴되지 않았다"며 주장과 함께 삭발 시위를 진행했다. 이날 도예인들은 도자기축제 행사 때 돔 텐트를 설치해 참여 도예인들이 한 공간에서 행사를 치르자는 입장이다. 관람객의 관심을 한 곳으로 집중시킬 수 있고 봄철 미세먼지와 바람에 취약한 날씨 여건 등을 해결할 수 있어 도자기 판매에 유리하다고 밝혔다. 하지만 시는 현재 예스파크 내에는 돔 텐트를 설치할 만큼의 공간이 부족하고, 지난 행사 때 미흡한 점 등을 보완하기 위한 방안으로 예스파크 내 위치한 건물 공실을 이용하자는 안을 고수하고 있다. 강봉구 이천도자기조합 상임이사는 "지난 축제 결과보고서를 토대로 개선책을 마련하던 중에 난데없이 그동안 논의했던 사항을 시가 협의를 배제한채 일방적으로 변경했다"며 "(축제 수정안과 관련)도예인이 반대하는 것은 축제 판매부스의 형태와 위치다. 판매부스는 관람객의 동선과 구매력에 직접적으로 영향을 미치기 때문에 도예인들이 가장 예민하게 생각하는 문제"라고 돔텐트 설치의 당위성을 주장했다. 이어 강 상임이사는 "민주주의 지방자치의 정신에 입각해 축제의 이해당사자와 시민 전문가로 구성된 축제추진위원회를 구성, 투명한 절차에 의해 모든 사항을 결정하자"고 대안을 제시했다. 이에 대해 시 관계자는 "축제 판매부스의 형태와 위치 등은 시가 일방적으로 결정한 것이 아니다"며라 "지난 축제의 문제점을 보완하기 위해 도예인 등 다방면 의견을 수렴해 결정된 사항"이라고 입장을 밝혔다.

## Train

In [ ]:
!wandb login

In [ ]:
import wandb

wandb.login()

In [17]:
learning_rate = 5e-6
optimizer = AdamW(model.parameters(), lr=learning_rate)

num_epochs = 2
num_training_steps = num_epochs * len(train_dataloader)

config = dict(
    epochs=num_epochs,
    batch_size=batch_size,
    learning_rate=learning_rate)

device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
model.to(device)
print("device :",device)

device : cuda


In [18]:
with wandb.init(project="<your_project>", config=config, resume=True):
    progress_bar = tqdm(range(num_training_steps))
    model.train()
    wandb.watch(model, log="all", log_freq=10)

    for epoch in range(2,4):
        total_loss = 0
        ### Train STEP
        for bindx, batch in enumerate(train_dataloader):
            input_ids, attention_mask, label = batch['input_ids'],batch['attention_mask'],batch['label']
            input_ids = input_ids.to(device)
            attention_mask = attention_mask.to(device)
            label = label.to(device)

            out = model.forward(input_ids=input_ids.flatten(1), 
                                attention_mask=attention_mask.flatten(1), 
                                labels=label.flatten(1)
                                )

            optimizer.zero_grad()
            out.loss.backward()
            optimizer.step()

            with torch.no_grad():
                progress_bar.update(1)
                loss = out.loss.item()
                total_loss += loss
                wandb.log({"epoch": epoch, "loss": loss, "batch":bindx, "total_loss":total_loss})


                if bindx%6250 ==0 and bindx!=0:

                    model.eval()
                    val_loss = 0
                    for vindx, batch in enumerate(val_dataloader):
                        input_ids, attention_mask, label,label_text = batch['input_ids'],batch['attention_mask'],batch['label'],batch['label_text']
                        input_ids = input_ids.to(device)
                        attention_mask = attention_mask.to(device)
                        label = label.to(device)
                        
                        out = model.forward(input_ids=input_ids.flatten(1), 
                                            attention_mask=attention_mask.flatten(1), 
                                            labels=label.flatten(1)
                                            )

                        val_loss += out.loss.item()
                    print(f'{epoch}epoch {bindx}batch','-'*100)
                    generated = model.generate(input_ids)
                    print("PRED  :",''.join(tokenizer.batch_decode(generated[0])))
                    print("LABEL :",batch['label_text'][0])
                    print("PRED  :",''.join(tokenizer.batch_decode(generated[1])))
                    print("LABEL :",batch['label_text'][1])

                    wandb.log({"val_loss": val_loss, "batch":bindx, "epoch": epoch})
                    torch.save(model.state_dict(), f"/<your_dir>/KoPEGASUS_Finetuning_{epoch}_{bindx}_{val_loss:.5f}.pt")
                    model.train()

        torch.save(model.state_dict(), f"/<your_dir>/KoPEGASUS_Finetuning_{epoch}_done_0.00000.pt")


  0%|          | 0/125000 [00:00<?, ?it/s]

2epoch 6250batch ----------------------------------------------------------------------------------------------------
PRED  : <usr>경북도와경주시,한국원자력연구원이16일경주시감포읍나정리에<usr>
LABEL : 경북도·경주시·한국원자력연구원은 16 일 경주시 감포읍 나정리에 들어설 1조원 규모의 에너지과학연구단지 조성을 위한 업무협약을 체결한 가운데 인·허가에서부터 완공 및 운영까지의 세월이 길면서 비용조달에 따른 시행착오가 있을 수 있어 약속한 모든 사항을 이른 시일 내에 이행해야 하며 에너지과학연구단지를 빠르게 완성할 수 있는 대책을 마련해야 한다.</s>
PRED  : <usr>부안군에대한전북도의종합감사에서결원을초과한승진인사를임용하는등<usr>
LABEL : 27일 전북도는 최근 부안군 종합감사에서 시정 8건, 주의 7건, 시정주의 14건, 시정개선 2건, 시정주의개선 1건 등 결원을 초과한 승진 인사 임용 등을 포함하는 총 32건의 부정 행위를 적발하고 훈계 27건과 회수 7억8000만원, 감액 7억8000만원, 추징 1억3000만원 등 총 17억원에 대한 재정조치를 취했다고 밝혔다.</s>
2epoch 12500batch ----------------------------------------------------------------------------------------------------
PRED  : <usr>경북도와경주시,한국원자력연구원이16일경주시감포읍나정리에<usr>
LABEL : 경북도·경주시·한국원자력연구원은 16 일 경주시 감포읍 나정리에 들어설 1조원 규모의 에너지과학연구단지 조성을 위한 업무협약을 체결한 가운데 인·허가에서부터 완공 및 운영까지의 세월이 길면서 비용조달에 따른 시행착오가 있을 수 있어 약속한 모든 사항을 이른 시일 내에 이행해야 하며 에너지과학연구단지를 빠르게 완성할 수 있는 대책을 마련해야 한다.</s>
P

batch,▁▁▂▂▂▃▃▃▄▄▅▅▅▆▆▆▇▇▇█▁▁▂▂▃▃▃▄▄▄▅▅▅▆▆▆▇▇██
epoch,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁████████████████████
loss,█▃▃▄▄▃▄▂▄▁▂▃▅▂▁▆▂▄▃▅▃▅▃▅▄▄▂█▄▃▅▄▄▄▄▃▁▃▄▅
total_loss,▁▁▂▂▂▃▃▄▄▄▅▅▅▆▆▇▇▇██▁▁▂▂▂▃▃▄▄▄▅▅▅▆▆▆▇▇▇█
val_loss,█▇▇▆▅▅▅▄▃▃▄▃▃▂▂▁▂▁
batch,62499
epoch,3
loss,0.11766
total_loss,9321.42949
val_loss,382.83268
